## TC 5033
### Word Embeddings

<br>

#### Activity 3b: Text Classification using RNNs and AG_NEWS dataset in PyTorch
<br>

- Objective:
    - Understand the basics of Recurrent Neural Networks (RNNs) and their application in text classification.
    - Learn how to handle a real-world text dataset, AG_NEWS, in PyTorch.
    - Gain hands-on experience in defining, training, and evaluating a text classification model in PyTorch.
    
<br>

- Instructions:
    - Data Preparation: Starter code will be provided that loads the AG_NEWS dataset and prepares it for training. Do not modify this part. However, you should be sure to understand it, and comment it, the use of markdown cells is suggested. 

    - Model Setup: A skeleton code for the RNN model class will be provided. Complete this class and use it to instantiate your model.

    - Implementing Accuracy Function: Write a function that takes model predictions and ground truth labels as input and returns the model's accuracy.

    - Training Function: Implement a function that performs training on the given model using the AG_NEWS dataset. Your model should achieve an accuracy of at least 80% to get full marks for this part.

    - Text Sampling: Write a function that takes a sample text as input and classifies it using your trained model.

    - Confusion Matrix: Implement a function to display the confusion matrix for your model on the test data.

    - Submission: Submit your completed Jupyter Notebook. Make sure to include a markdown cell at the beginning of the notebook that lists the names of all team members. Teams should consist of 3 to 4 members.
    
<br>

- Evaluation Criteria:

    - Correct setup of all the required libraries and modules (10%)
    - Code Quality (30%): Your code should be well-organized, clearly commented, and easy to follow. Use also markdown cells for clarity. Comments should be given for all the provided code, this will help you understand its functionality.
    
   - Functionality (60%): 
        - All the functions should execute without errors and provide the expected outputs.
        - RNN model class (20%)
        - Accuracy fucntion (10%)
        - Training function (10%)
        - Sampling function (10%)
        - Confucion matrix (10%)

        - The model should achieve at least an 80% accuracy on the AG_NEWS test set for full marks in this criterion.


Dataset

https://pytorch.org/text/stable/datasets.html#text-classification

https://paperswithcode.com/dataset/ag-news


### Import libraries

In [1]:
# conda install -c pytorch torchtext
# conda install -c pytorch torchdata
# conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch

In [2]:
# The following libraries are required for running the given code
# Please feel free to add any libraries you consider adecuate to complete the assingment.
import numpy as np
#PyTorch libraries
import torch
from torchtext.datasets import AG_NEWS
# Dataloader library
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
# Libraries to prepare the data
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
# neural layers
from torch import nn
from torch.nn import functional as F

# These libraries are suggested to plot confusion matrix
# you may use others
#import scikitplot as skplt
#import gc

In [3]:
# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


### Get the train and the test datasets and dataloaders

Classes:

* 1 - World

* 2 - Sports

* 3 - Business

* 4 - Sci/Tech

We will convert them to:

* 0 - World

* 1 - Sports

* 2 - Business

* 3 - Sci/Tech

In [4]:
train_dataset, test_dataset = AG_NEWS()
train_dataset, test_dataset = to_map_style_dataset(train_dataset), to_map_style_dataset(test_dataset)

In [5]:
# Get the tokeniser
# tokeniser object
tokeniser = get_tokenizer('basic_english')

def yield_tokens(data):
    for _, text in data:
        yield tokeniser(text)

In [6]:
# Build the vocabulary
vocab = build_vocab_from_iterator(yield_tokens(train_dataset), specials=["<unk>"])
#set unknown token at position 0
vocab.set_default_index(vocab["<unk>"])

In [7]:
#test tokens
tokens = tokeniser('Welcome to TE3007')
print(tokens, vocab(tokens))

['welcome', 'to', 'te3007'] [3314, 4, 0]


In [8]:
NUM_TRAIN = int(len(train_dataset)*0.9)
NUM_VAL = len(train_dataset) - NUM_TRAIN

In [9]:
train_dataset, val_dataset = random_split(train_dataset, [NUM_TRAIN, NUM_VAL])

In [10]:
print(len(train_dataset), len(val_dataset), len(test_dataset))

108000 12000 7600


In [11]:
# function passed to the DataLoader to process a batch of data as indicated
def collate_batch(batch):
    # Get label and text
    y, x = list(zip(*batch))
    
    # Create list with indices from tokeniser
    x = [vocab(tokeniser(text)) for text in x]
    x = [t + ([0]*(max_tokens - len(t))) if len(t) < max_tokens else t[:max_tokens] for t in x]

    # Prepare the labels, by subtracting 1 to get them in the range 0-3
    return torch.tensor(x, dtype=torch.int32), torch.tensor(y, dtype=torch.int32) - 1

In [12]:
labels =  ["World", "Sports", "Business", "Sci/Tech"]
max_tokens = 50
BATCH_SIZE = 256

In [13]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch, shuffle = True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, collate_fn=collate_batch, shuffle = True)

### Let us build our RNN model

In [14]:
EMBEDDING_SIZE = 300# complete
NEURONS = 256# complete
LAYERS = 2# complete
NUM_CLASSES = 4# complete

In [ ]:
# Define a class for the RNN model which inherits from nn.Module.
class RNN_Model_1(nn.Module):
    # Sets up the model architecture with the given parameters.
    def __init__(self, embed_size, hidden, layers, num_classes):

         # Initialize the superclass nn.Module.
        super().__init__()

        # Turns word indexes into embeddings of specified size.
        # len(vocab) specifies the total number of embeddings to create, one for each word.
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab),
                                            embedding_dim=embed_size)

        # A GRU model used
        # input_size = size of the input
        # hidden_size = size of the model hidden section
        # num_layers = number of stacked GRU layers.
        # batch_first = indicates that the input will have the batch dimension first
        # bidirectional = the GRU will process the input in both directions
        self.rnn = nn.GRU(input_size = embed_size,
                          hidden_size = hidden,
                          num_layers = layers,
                          batch_first = True,
                          bidirectional= True)

        # FCN that maps the RNN outputs to the number of classes
        # 2*hidden is used because the GRU is bidirectional, so it concatenates the hidden states from both directions
        self.fc = nn.Linear(in_features=2*hidden,out_features=num_classes)

    # Fordward propagation function
    def forward(self, x):

        # Pass the input through the embedding layer
        vector_embs = self.embedding_layer(x)

        # Pass the embeddings to the GRU layer
        # 'y' are the output features from the last layer of the GRU for each timestep
        # 'h' are the hidden states for each layer at the last timestep
        y,h = self.rnn(vector_embs)

        # Pass the final output features through the fully connected layer
        # selects the features from the last timestep of the GRU model
        return self.fc(y[:,-1])

In [ ]:
# Define a function to calculate accuracy
def accuracy(model, loader):

    #Switch the model to evaluation mode
    model.eval()

    # Move the model to the appropriate device CPU or GPU
    model = model.to(device = device)

    # Initialize 'correct' as a counter for the number of correct predictions
    # Initialize  'total' as a counter for the total number of samples processed
    correct = 0
    total = 0

    # Disable gradient computation not needed during model evaluation
    with torch.no_grad():

        # Loop over all batches of data in loader.
        for data in loader:

            # Loads input data and labels in the device
            inputs, labels = data[0].to(device), data[1].to(device)

             # Pass the input data to the model to get predictions
            outputs = model(inputs)

            # Search the class with the highest predicted probability
            _, predicted = torch.max(outputs, 1)

            # Increment the total count by the number of samples in the current batch
            total += labels.size(0)

            # Increment the correct count by the number of accurately predicted samples in the current batch
            correct += (predicted == labels).sum().item()

    # Calculate the accuracy as the percentage of correct predictions out of all samples
    accuracy = 100 * correct / total
    return accuracy

In [ ]:
def accuracyLucero(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_tensor = predicted.eq(labels.data.view_as(predicted))
            accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
    return accuracy

In [ ]:
def accuracyLucero(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_tensor = predicted.eq(labels.data.view_as(predicted))
            accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
    return accuracy

In [ ]:
def accuracyLucero(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_tensor = predicted.eq(labels.data.view_as(predicted))
            accuracy = torch.mean(
                        correct_tensor.type(torch.FloatTensor))
    return accuracy

In [ ]:
train(gru_model, optimiser=optimiser,  epochs=epochs)

In [ ]:
print(f'{accuracy(gru_model, test_loader):.4f}')

In [ ]:
def sample_text(model, loader):
    pass

In [ ]:
sample_text(rnn_model, test_loader)

In [ ]:
# create confusion matrix
pass